In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import math
from scipy import optimize

import matplotlib as mpl
from matplotlib import cm
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['mathtext.rm'] = 'serif'

import turning_point as tp
import gezeltermiller_hamiltonian as gm

import os
path_to_data = 'data/'
path_to_saveplot = 'plots/'

In [7]:
#%% Setting up parameters and global variables

save_final_plot = True
show_final_plot = False
show_itrsteps_plots = False # show iteration of the UPOs in plots
N = 4         # dimension of phase space

masses = [16, 1]
params_pe = [
    1.0074e-2,  # k
    1.9769e0,   # d
    -2.3597e-3, # a2
    1.0408e-3,  # a4
    -7.5496e-5, # a6
    7.7569e-3,  # c
    -2.45182e-4 # d0
            ]

parameters = masses + params_pe
eqNum = 3
eqPt = tp.get_eq_pts(eqNum, gm.init_guess_eqpt_gm, \
                    gm.grad_pot_gm, parameters)


In [8]:
eqPt

array([-0.54669686,  0.00217407])

In [4]:
#%% 
#E_vals = [1.1, 2.00, 3.00, 5.00]
#linecolor = ['b','r','g','m','c']
E_vals = [0.01, 0.02]
linecolor = ['b','r']

n = 4 # number of intervals we want to divide
n_turn = 1 # nth turning point we want to choose.
    
for i in range(len(E_vals)):
    
    e = E_vals[i] # total energy
        
    #Trial initial Condition s.t. one initial condition is on the LHS of the UPO and the 
    #other one is on the RHS of the UPO

    f1 = lambda x: gm.get_coord_gm(x,1.51,e,parameters)
    x0_2 = optimize.newton(f1,-3)
    state0_2 = [x0_2,1.51,0.0,0.0]

    f2 = lambda x: gm.get_coord_gm(x,1.49,e,parameters)
    x0_3 = optimize.newton(f2,-3)
    state0_3 = [x0_3,1.49,0.0,0.0]
    
    with open("x0_turningpoint_e%s_gm.dat"%(e),'a+') as po_fam_file:
    
        [x0po_1, T_1,energyPO_1] = tp.turningPoint( 
            state0_2, state0_3, [0, 200], gm.get_coord_gm, gm.guess_coords_gm, gm.ham2dof_gm, \
            gm.half_period_gm, gm.variational_eqns_gm, gm.pot_energy_gm, \
            gm.plot_iter_orbit_gm, 
            parameters, e, n, n_turn, show_itrsteps_plots, po_fam_file) 

h is  -0.0
Initial guess1[-3.3384251199919714, 1.51, 0.0, 0.0], initial guess2[-3.3384251199920185, 1.51, 0, 0], dot product is0.00012785450729364538
h is  -0.0050000000000000044
Initial guess1[-3.3384251199919714, 1.51, 0.0, 0.0], initial guess2[-3.326679249905633, 1.505, 0, 0], dot product is0.00012258406529502656
h is  -0.010000000000000009
Initial guess1[-3.3384251199919714, 1.51, 0.0, 0.0], initial guess2[-3.3148277926241008, 1.5, 0, 0], dot product is0.00011636157928449599
h is  -0.015000000000000013
Initial guess1[-3.3384251199919714, 1.51, 0.0, 0.0], initial guess2[-3.302883627484259, 1.495, 0, 0], dot product is0.00011130949348140967
h is  -0.020000000000000018
Initial guess1[-3.3384251199919714, 1.51, 0.0, 0.0], initial guess2[-3.290861668572854, 1.49, 0, 0], dot product is0.00010635707867898855


UnboundLocalError: local variable 'check_same' referenced before assignment

In [ ]:
#%% Load periodic orbit data from ascii files
    
x0po = np.zeros((4,len(E_vals))) #each column is a different initial condition

for i in range(len(E_vals)):
    
    e = E_vals[i]
    deltaE = e - parameters[2]

    with open("x0_turningpoint_deltaE%s_deleonberne.dat"%(deltaE),'a+') as po_fam_file:
        print('Loading the periodic orbit family from data file',po_fam_file.name,'\n') 
        x0podata = np.loadtxt(po_fam_file.name)
        x0po[:,i] = x0podata[-1,0:4] 

In [ ]:
#%% Plotting the family

TSPAN = [0,30]
plt.close('all')
axis_fs = 15
RelTol = 3.e-10
AbsTol = 1.e-10

f = lambda t,x : deleonberne.ham2dof_deleonberne(t,x,parameters) 

ax = plt.gca(projection='3d')

for i in range(len(E_vals)):
    
    e = E_vals[i]
    deltaE = e - parameters[2]
    
    soln = solve_ivp(f, TSPAN, x0po[:,i], method='RK45', dense_output=True, \
                    events = lambda t,x : deleonberne.half_period_deleonberne(t,x,parameters), \
                    rtol=RelTol, atol=AbsTol)
    
    te = soln.t_events[0]
    tt = [0,te[2]]
    t,x,phi_t1,PHI = tp.state_transit_matrix(tt, x0po[:,i], parameters, \
                                        deleonberne.variational_eqns_deleonberne)
    
    ax.plot(x[:,0],x[:,1],x[:,2],'-',color=linecolor[i], \
            label='$\Delta E$ = %.2f'%(deltaE))
    ax.scatter(x[0,0],x[0,1],x[0,2],s=10,marker='*')
    ax.plot(x[:,0], x[:,1], zs=0, zdir='z')

    
resX = 100
xVec = np.linspace(-1,1,resX)
yVec = np.linspace(-2,2,resX)
xMat, yMat = np.meshgrid(xVec, yVec)
cset1 = ax.contour(xMat, yMat, tp.get_pot_surf_proj(xVec, yVec, \
                    deleonberne.pot_energy_deleonberne, parameters), \
                    [0.01,0.1,1,2,4], zdir='z', offset=0, \
                    linewidths = 1.0, cmap=cm.viridis, \
                    alpha = 0.8)

ax.scatter(eqPt[0], eqPt[1], s = 50, c = 'r', marker = 'X')
ax.set_xlabel('$x$', fontsize=axis_fs)
ax.set_ylabel('$y$', fontsize=axis_fs)
ax.set_zlabel('$p_x$', fontsize=axis_fs)

ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_zlim(-4, 4)
legend = ax.legend(loc='upper left')

plt.grid()

if show_final_plot:
    plt.show()

if save_final_plot:  
    plt.savefig(path_to_saveplot + 'tp_deleonberne_upos.pdf', \
                format='pdf', bbox_inches='tight')
